# Prorrata ERNC
Este programa tiene por objetivo realizar el re-calculo de curtailment para el sistema, en base a metodología propuesta por la NTCyO.

## LECTURA DE DATOS
Los datos deben ser extraidos desde el accdb en potencia neta (no por la potencia, sino por los marginales no truncados). La lista de datos que se deben extraer son:
1. Generación de cada central.
2. Perfil de generación de cada central.
3. Barra asociada a cada central.
4. Costos marginales para cada.
5. Curtailment por central (quizas por barra es suficiente).
6. Potencia máxima.
7. Generación disponible.
8. Estado de operación.

Para la lectura, a modo de determinar la mejor query al sistema, sin tener que lidiar con los problemas de MS Access, se cargan las tablas en DuckDB y se utiliza jupysql para probar SQL.

In [ ]:
import polars as pl
import duckdb as duck

from pathlib import Path
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import (
    engine,
    create_engine,
    inspect
)

path_prg = Path(r"../data/Model PRGdia_Full_Definitivo Solution.accdb").absolute()

if not path_prg.exists():
    raise ValueError(f"Path: {path_prg} does not exists.")

connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={path_prg.as_posix()};"
    r"ExtendedAnsiSQL=1;"
)
connection_url = engine.URL.create(
    "access+pyodbc",
    query={"odbc_connect": connection_string}
)

try:
    prg_engine = create_engine(connection_url)
    tables = inspect(prg_engine).get_table_names()

    conn = duck.connect("PCP.duckdb")
    #conn.execute("CREATE SCHEMA IF NOT EXISTS bronze")

    for table in tables:
        print(f"trabajando en tabla: {table}...")
        df = pl.read_database(query=f"SELECT * FROM {table}", connection=prg_engine)
        conn.execute(f"CREATE OR REPLACE TABLE {table} AS SELECT * FROM df")

except SQLAlchemyError as e:
    print(f"Error: {e}")

finally:
    conn.close()
    prg_engine.dispose()

## REVISIÓN DUCKDB
Con la data carga en la base de datos, empezamos a mirar como armar la mejor query

In [ ]:
# Esto es para carga la extensión y leer la base de datos.
import duckdb

conn_pcp = duckdb.connect("pcp.duckdb")

# load de la extensión para sql
%load_ext sql
%sql conn_pcp --alias duck

In [ ]:
# Esto es para cerrar las conexiones, usarlo al terminar de revisar
%sql --close duck
conn.close()

Clasica query para sacar la generación, levemente modificada para sacar las otras propiedades de una, no es necesario tener una query por dato.

In [ ]:
%%sql
SELECT 
    t_child.name AS generator,
    t_property.name AS property,
    t_period_0.datetime,
    t_data_0.key_id AS data_key,
    t_data_0.period_id AS data_period,
    t_data_0.value,
FROM ((((((((t_membership
INNER JOIN t_collection ON t_membership.collection_id = t_collection.collection_id)
INNER JOIN t_object AS t_parent ON t_membership.parent_object_id = t_parent.object_id)
INNER JOIN t_object AS t_child ON t_membership.child_object_id = t_child.object_id)
INNER JOIN t_property ON t_collection.collection_id = t_property.collection_id)
INNER JOIN t_key ON t_membership.membership_id = t_key.membership_id AND t_property.property_id = t_key.property_id)
INNER JOIN t_data_0 ON t_key.key_id = t_data_0.key_id)
INNER JOIN t_phase_3 ON t_data_0.period_id = t_phase_3.period_id)
INNER JOIN t_period_0 ON t_phase_3.interval_id = t_period_0.interval_id)
INNER JOIN t_category ON t_child.category_id = t_category.category_id
WHERE t_collection.collection_id = 1 AND t_property.property_id IN (1, 28, 200, 219) AND t_category.category_id IN (95, 96, 99, 100)

Similar a la anterior, una query para sacar los datos con marginales negativos, no se necesita el resto.

In [ ]:
%%sql
SELECT 
    t_child.name AS node,
    t_period_0.datetime,
    t_data_0.key_id AS data_key,
    t_data_0.period_id AS data_period,
    t_data_0.value AS marginal_cost,
FROM ((((((((t_membership
INNER JOIN t_collection ON t_membership.collection_id = t_collection.collection_id)
INNER JOIN t_object AS t_parent ON t_membership.parent_object_id = t_parent.object_id)
INNER JOIN t_object AS t_child ON t_membership.child_object_id = t_child.object_id)
INNER JOIN t_property ON t_collection.collection_id = t_property.collection_id)
INNER JOIN t_key ON t_membership.membership_id = t_key.membership_id AND t_property.property_id = t_key.property_id)
INNER JOIN t_data_0 ON t_key.key_id = t_data_0.key_id)
INNER JOIN t_phase_3 ON t_data_0.period_id = t_phase_3.period_id)
INNER JOIN t_period_0 ON t_phase_3.interval_id = t_period_0.interval_id)
INNER JOIN t_category ON t_child.category_id = t_category.category_id
WHERE t_collection.collection_id = 245 AND t_property.property_id = 1233 AND t_data_0.value < 0

Query bonita con CTE para extraer la relación entre barra y generador. Lamentablemente no hay CTE en MSACCESS por lo que se reformula en el .sql

In [ ]:
%%sql
WITH node_obj AS (
    SELECT 
        t_object.object_id AS node_id,
        t_object.name AS node,
    FROM t_object
    INNER JOIN t_class ON t_object.class_id = t_class.class_id
    WHERE t_class.name = 'Node'
), gen_obj AS (
    SELECT 
        t_object.object_id AS gen_id,
        t_object.name AS generator,
    FROM t_object
    INNER JOIN t_class ON t_object.class_id = t_class.class_id
    WHERE t_class.name = 'Generator' AND t_object.category_id IN (95, 96, 99, 100)
)

SELECT
    node_obj.node,
    gen_obj.generator,
FROM t_membership
INNER JOIN node_obj ON t_membership.child_object_id = node_obj.node_id
INNER JOIN gen_obj ON t_membership.parent_object_id = gen_obj.gen_id
WHERE t_membership.collection_id = 12

## Juntando la información
Con el trabajo de armar los SQL, ahora se pasa a solo usar polars para disminuir la necesidad de otra libreria `DuckDB` (por mucho que me guste esta db).

In [ ]:
import polars as pl

from pathlib import Path
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import (
    engine,
    create_engine,
)

# Lectura de SQL Barra-Generador
path_sql_node = Path(r"../poc_prorrataerv/sql/gen_node.sql").absolute()
with open(path_sql_node, "r") as file:
    sql_node = file.read()

# Lectura de SQL con data de generacion
path_sql_gen = Path(r"../poc_prorrataerv/sql/gen_data.sql").absolute()
with open(path_sql_gen, "r") as file:
    sql_gen = file.read()

# lectura de SQL con data de barras con costos marginales menor a 0
path_sql_cmg = Path(r"../poc_prorrataerv/sql/cmg_data.sql").absolute()
with open(path_sql_cmg, "r") as file:
    sql_cmg = file.read()

# Inicio de captura de datos en dataframes
path_prg = Path(r"../data/Model PRGdia_Full_Definitivo Solution.accdb").absolute()

if not path_prg.exists():
    raise ValueError(f"Path: {path_prg} does not exists.")

connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={path_prg.as_posix()};"
    r"ExtendedAnsiSQL=1;"
)
connection_url = engine.URL.create(
    "access+pyodbc",
    query={"odbc_connect": connection_string}
)

try:
    prg_engine = create_engine(connection_url)

    df_nodes = pl.read_database(query=sql_node, connection=prg_engine)
    df_gen = pl.read_database(query=sql_gen, connection=prg_engine)
    df_cmg = pl.read_database(query=sql_cmg, connection=prg_engine)

except SQLAlchemyError as e:
    print(f"Error: {e}")

finally:
    prg_engine.dispose()


In [ ]:
# Lectura de otros datos pmgd
path_pmgd = Path(r"W:/41 Dpto Pronosticos/Vertimiento_ERNC/Lista_PMGDs.xlsx").absolute()
df_pmgd = pl.read_excel(
    source=path_pmgd,
    sheet_name="Hoja1",
    xlsx2csv_options={"skip_empty_lines": True},
    read_csv_options={"new_columns": ["Nombre_CDC","Centrales"]},
)

# lectura de lista de centrales vetadas
path_vetados = Path(r"R:/Aplicaciones/Prorrateo_Vertimiento/Centrales_Vetadas.xlsx").absolute()
df_vetados = pl.read_excel(
    source=path_vetados,
    sheet_name="Hoja1",
    xlsx2csv_options={"skip_empty_lines": True},
    read_csv_options={"new_columns": ["Centrales"]},
)

In [ ]:
# Check errores de curtailement por centrales no pintadas en rojo.
(
    df_gen
    .filter(
        pl.col("generator").is_in(
            pl.concat(
                [df_vetados["Centrales"].unique(),
                 df_pmgd["Centrales"].unique()]
            )
        )
    )
    .pivot(
        values="value",
        columns="property",
        index=["generator", "datetime"]
    )
    .filter(
        pl.col("Units Generating") == 1,
        pl.col("Capacity Curtailed") != 0,
    )
    .select(
        pl.exclude("Units Generating")
    )
    .group_by(pl.col("generator").alias("Generator"))
    .agg(pl.col("Capacity Curtailed").sum().alias("Total Capacity Curtailed"))
)

In [ ]:
# transformación de data para generación, eliminando centrales vetadas y pmgd
# y pivotear la data para tener las columnas de las propiedades
# y filtrar las centrales que no estan generando
df_gen_pivot = (
    df_gen
    .filter(
        ~pl.col("generator").is_in(df_vetados["Centrales"].unique()),
        ~pl.col("generator").is_in(df_pmgd["Centrales"].unique()),
    )
    .pivot(
        values="value",
        columns="property",
        index=["generator", "datetime"]
    )
    .filter(
        pl.col("Units Generating") == 1,
    )
    .select(
        pl.exclude("Units Generating")
    )
)
df_gen_pivot

In [ ]:
def calc_error(df: pl.LazyFrame, original_col: str = "Generation", prorrata_col: str = "Prorrata", over_col: str = "datetime") -> pl.LazyFrame:
    return (
        df
        .with_columns(
            pl.when(pl.col("Prorrata").lt(0))
            .then(pl.col("Prorrata").abs())
            .otherwise(0)
            .alias("Error"),
            pl.when(pl.col("Prorrata").lt(0))
            .then(0)
            .otherwise(pl.col("Prorrata"))
            .alias("Prorrata"),
        )
    )

def check_error(df: pl.LazyFrame, error_col: str = "Error", tol: float = 1e-3) -> bool:
    return df.select(pl.col(error_col).ge(tol).any()).collect().item()

def show_total_error(df: pl.LazyFrame, error_col: str = "Error") -> float:
    return df.select(pl.col(error_col).sum().alias("Total Error")).collect().item()

def calc_prorrata(df: pl.LazyFrame, target_col: str = "Prorrata", error_col: str = "Error", weight_col: str = "Max Capacity", over_col: str = "datetime") -> pl.LazyFrame:
    return (
        df
        .with_columns(
            (pl.col(target_col) - pl.col(error_col).sum().over(over_col) * pl.col(weight_col) / pl.col(weight_col).sum().over(over_col)).alias("Prorrata"),
        )
    )

In [ ]:
def process_prorrata(df: pl.LazyFrame, target_col: str = "Prorrata", error_col: str = "Error", weight_col: str = "Max Capacity", over_col: str = "datetime") -> pl.LazyFrame:
    df_processed = calc_prorrata(df,target_col,error_col)
    df_processed = calc_error(df_processed)

    print(check_error(df_processed))
    print(show_total_error(df_processed))

    if check_error(df_processed):
        return process_prorrata(df_processed)
    
    return df_processed

In [ ]:
test_data = (
    df_cmg
    .join(df_nodes, on="node", how="inner")
    .join(df_gen_pivot, on=["generator","datetime"], how="inner")
)

In [ ]:
test_prorrata_func = process_prorrata(test_data.lazy(),"Available Capacity","Capacity Curtailed")

In [ ]:
test_prorrata_func.collect().describe()

In [ ]:
(
    test_prorrata_func
    .sort(by="datetime")
    .group_by("datetime")
    .agg(
        #pl.col("Prorrata_Curt").sum().alias("Total_Curatiled"),
        pl.col("Generation").sum().alias("Total_Gen"),
        pl.col("Prorrata").sum().alias("Total_Gen_Prorrata"),
        #pl.col("Prorrata").min().alias("Min_Gen_Prorrata"),
        #pl.col("Prorrata").filter(pl.col("Prorrata").lt(0)).sum().alias("Sum_Gen_Prorrata"),
        pl.col("Error").sum().alias("Total_Error"),
        (pl.col("Prorrata") - pl.col("Error")).sum().alias("Sum_Prorrata_error"),
        (pl.col("Generation") - (pl.col("Prorrata") - pl.col("Error"))).sum().alias("Test_total"),
    )
).collect()

In [ ]:
test_prorrata_func.collect()

In [ ]:
df_gen

In [ ]:
data_to_update = (
    test_prorrata_func
    .join(
        df_gen.filter(pl.col("property") == "Generation").lazy(),
        on=["generator","datetime"],
        how="inner"
    )
    .select(
        #pl.col("generator"),
        #pl.col("datetime"),
        #pl.col("Generation"),
        #pl.col("value").alias("Generation_Ori"),
        pl.col("data_key").alias("key_id"),
        pl.col("data_period").alias("period_id"),
        pl.col("Prorrata").alias("value"),
    )
    .sort(by=["key_id","period_id"])
    .collect()
)
data_to_update

In [ ]:
test_dict = data_to_update.head().to_dicts()
for row in test_dict:
    print(row)

In [ ]:
from sqlalchemy import (
    engine,
    create_engine,
    inspect,
)
from sqlalchemy.sql.expression import (
    update,
    table,
    column,
)

from pathlib import Path

import polars as pl

df_data = pl.read_csv("../t_data_0.csv")

t_data_0 = table("t_data_0", column("key_id"), column("period_id"), column("value"))
#stmt = update(t_data_0).where(t_data_0.c.key_id == 1).values(value=0.0)

# Inicio de captura de datos en dataframes
path_prg = Path(r"../data/Model PRGdia_Full_Definitivo Solution_neto.accdb").absolute()

if not path_prg.exists():
    raise ValueError(f"Path: {path_prg} does not exists.")

connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={path_prg.as_posix()};"
    r"ExtendedAnsiSQL=1;"
)
connection_url = engine.URL.create(
    "access+pyodbc",
    query={"odbc_connect": connection_string}
)

prg_engine = create_engine(connection_url)
tables = inspect(prg_engine).get_table_names()

with prg_engine.begin() as conn:
    for row in df_data.head().to_dicts():
        stmt = (
            t_data_0.update()
            .where(t_data_0.c.key_id == row.get("key_id"))
            .where(t_data_0.c.period_id == row.get("period_id"))
            .values(value=row.get("value"))
        )
        restuls = conn.execute(stmt)
        print(restuls.)
    #conn.commit()


In [ ]:
import pyodbc
import polars as pl
from pathlib import Path

df_data = pl.read_csv("../t_data_0.csv")
to_update = df_data.head().to_dicts()


path_prg = Path(r"../data/Model PRGdia_Full_Definitivo Solution_neto.accdb").absolute()

connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={path_prg.as_posix()};"
    r"ExtendedAnsiSQL=1;"
)

cnxn = pyodbc.connect(connection_string)
crsr = cnxn.cursor()

#sql = "SELECT TOP 10 * FROM t_data_0 "
#update_sql = r"UPDATE t_data_0 SET t_data_0.value = ? WHERE t_data_0.key_id = ? AND t_data_0.period_id = ?;"
#params = [(data['value'],data['key_id'],data['period_id']) for data in to_update]
#crsr.execute(update_sql, params)
#crsr.commit()
#cnxn.close()

try:
    cnxn.autocommit = False
    params = params = [(data['value'],data['key_id'],data['period_id']) for data in to_update]
    crsr.executemany("UPDATE t_data_0 SET t_data_0.value = ? WHERE t_data_0.key_id = ? AND t_data_0.period_id = ?;", params)
except pyodbc.DatabaseError as err:
    cnxn.rollback()
else:
    cnxn.commit()
finally:
    cnxn.close()

In [ ]:
[(data['value'],data['key_id'],data['period_id']) for data in to_update]

In [ ]:
df_data.head().to_dict(as_series=False)['key_id']

In [ ]:
from pathlib import Path
path_prg = Path(r"../data/Model PRGdia_Full_Definitivo Solution_neto.accdb").absolute()

connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={path_prg.as_posix()};"
    r"ExtendedAnsiSQL=1;"
)
connection_string

## Cambio de criterio
Ajutando código para cambiar como se calcula.

In [1]:
from pathlib import Path
from prorrata.extract import DataExtractor
from prorrata.transform_cdc import DataProcessor
from prorrata.load import DataLoader


path_prg = r"C:\Users\felipe.bastidas\PyProyectos\Test_data\PRG20240214"
#PATH_PMGD_EXCLUDE =  r"C:\Users\felipe.bastidas\PyProyectos\Test_data\Otros\Lista_PMGDs.xlsx"
#PATH_BANNED_GENERATORS = r"C:\Users\felipe.bastidas\PyProyectos\Test_data\Otros\Centrales_Vetadas.xlsx"
PATH_ACCDB_INPUT = r"Datos/Model PRGdia_Full_Definitivo Solution/Model PRGdia_Full_Definitivo Solution.accdb"
PATH_ACCDB_OUTPUT = r"Antecedentes/Model PRGdia_Full_Definitivo Solution.accdb"
PATH_PMGD_EXCLUDE =  r"W:/41 Dpto Pronosticos/Vertimiento_ERNC/Lista_PMGDs.xlsx"
PATH_BANNED_GENERATORS = r"R:/Aplicaciones/Prorrateo_Vertimiento/Centrales_Vetadas.xlsx"

data_extractor = DataExtractor(
    Path(path_prg).joinpath(PATH_ACCDB_INPUT),
    Path(PATH_PMGD_EXCLUDE),
    Path(PATH_BANNED_GENERATORS)
)
data_extractor.extract_data()

data_processor = DataProcessor.from_extractor(data_extractor)
data_processor.process_prorrata()
data_processor.get_t_data(data_extractor)

data_loader = DataLoader(Path(path_prg).joinpath(PATH_ACCDB_OUTPUT))
data_loader.load_data(data_processor)

data_processor.show_results()

error: 1989.0003813397936
error: 953.2747084488323
error: 509.5545181783241
error: 296.04320680935473
error: 187.877467713414
error: 125.52106655354413
error: 90.03033471975382
error: 70.02642788214527
error: 57.403873081881265
error: 48.794183700691164
error: 42.96230778214258
error: 38.43857801074162
error: 34.75968040917212
error: 31.653898086065873
error: 28.95792941882811
error: 26.57081067231086
error: 24.427991063612904
error: 22.486538500227798
error: 20.716587762666663
error: 19.09633783127833
error: 17.88069350595735
error: 17.168356955635236
error: 16.597949073459368
error: 16.048329041420164
error: 15.647051090034584
error: 15.427854546601262
error: 15.212192665176183
error: 14.999829462608142
error: 14.790604873525274
error: 14.58440536295977
error: 14.381146034830723
error: 14.180759721373356
error: 13.983190319375375
error: 13.788388714032568
error: 13.596310281718441
error: 13.406913357418201
error: 13.220158292242543
error: 13.036006872282108
error: 12.854421958965466


datetime,Error_Prorrata,Total_Curtailed
datetime[μs],f64,f64
2024-02-14 10:00:00,-0.014271,104.54334
2024-02-14 11:00:00,-4.5475e-13,357.124528
2024-02-14 12:00:00,4.5475e-13,383.376433
2024-02-14 13:00:00,0.0,628.575549
2024-02-14 14:00:00,0.0,504.957896
2024-02-14 15:00:00,9.0949e-13,657.801
2024-02-14 16:00:00,0.0,549.73347
2024-02-14 17:00:00,0.0,311.638823


In [ ]:
import polars as pl
data_processor.data.select(pl.col("Error").sum()).collect().item()

In [ ]:
import polars as pl

WEIGHT_COL = "Max Capacity"
DATETIME_COL = "datetime"
GENERATION_COL = "Generation"
PRORRATA_COL = "Prorrata"
ERROR_COL = "Error"
AVAILABLE_COL = "Available Capacity"
ACTIVE_COL = "Active"

data = data_processor.data.with_columns(
            (
                pl.col(GENERATION_COL).sum().over(DATETIME_COL)
                * pl.col(WEIGHT_COL)
                / pl.col(WEIGHT_COL).sum().over(DATETIME_COL)
            ).alias(PRORRATA_COL),
            pl.lit(0).alias(ERROR_COL)
        ).collect()
data.limit(5)

In [ ]:
# ERROR
(
    data
    .with_columns(
        pl.when(pl.col(PRORRATA_COL).gt(pl.col(AVAILABLE_COL)))
        .then(pl.col(PRORRATA_COL) - pl.col(AVAILABLE_COL))
        .otherwise(0)
        .alias(ERROR_COL),

        pl.when(pl.col(PRORRATA_COL).gt(pl.col(AVAILABLE_COL)))
        .then(pl.col(AVAILABLE_COL))
        .otherwise(pl.col(PRORRATA_COL))
        .alias(PRORRATA_COL),
    )
)
data.limit(5)

In [ ]:
# new prorrata
(
    data
    .with_columns(
        (
            pl.col(PRORRATA_COL)
            + pl.col(ERROR_COL).sum().over(DATETIME_COL)
            * pl.col(WEIGHT_COL)
            / pl.col(WEIGHT_COL).sum().over(DATETIME_COL)
        ).alias(PRORRATA_COL),
    )
)
data.limit(5)

In [29]:
from pathlib import Path
from prorrata.extract import DataExtractor


path_prg = r"C:\Users\felipe.bastidas\PyProyectos\Test_data\PRG20240702"
#PATH_PMGD_EXCLUDE =  r"C:\Users\felipe.bastidas\PyProyectos\Test_data\Otros\Lista_PMGDs.xlsx"
#PATH_BANNED_GENERATORS = r"C:\Users\felipe.bastidas\PyProyectos\Test_data\Otros\Centrales_Vetadas.xlsx"
PATH_ACCDB_INPUT = r"Datos/Model PRGdia_Full_Definitivo Solution/Model PRGdia_Full_Definitivo Solution.accdb"
PATH_PMGD_EXCLUDE =  r"W:/41 Dpto Pronosticos/Vertimiento_ERNC/Lista_PMGDs.xlsx"
PATH_BANNED_GENERATORS = r"R:/Aplicaciones/Prorrateo_Vertimiento/Centrales_Vetadas.xlsx"

data = DataExtractor(
    Path(path_prg).joinpath(PATH_ACCDB_INPUT),
    Path(PATH_PMGD_EXCLUDE),
    Path(PATH_BANNED_GENERATORS)
)

data.extract_data()

In [31]:
(
    data.pmgd
    .filter(pl.col("Centrales").str.contains("_EO"))
)

Nombre_CDC,Centrales
str,str
null,"""CABO_NEGRO_EO"""
null,"""VIENTOS_PATAGO…"
null,"""LEBU_EO"""
null,"""EL_ARREBOL_EO"""
null,"""EL_NOGAL_EO"""
null,"""HUAJACHE_EO"""
null,"""LAS_PENAS_EO"""
null,"""LEBU_3_EO"""
null,"""RAKI_EO"""


In [39]:
(
    data.nodes
    .filter(pl.col("generator").eq("MALLECO_SUR_EO"))
)

generator,node
str,str
"""MALLECO_SUR_EO…","""Mulchen220"""


In [41]:
(
    data.cmg
    .filter(pl.col("node").eq("Mulchen220"))
)

node,datetime
str,datetime[μs]


In [36]:
(
    data.gen
    #.filter(
    #    pl.col("property").is_in(["Capacity Curtailed","Units Generating"]),
    #    #pl.col("value").gt(0)
    #)
    #.group_by(by=pl.col("datetime")).agg(pl.col("value").sum())
    .pivot(values="value",index=["datetime","generator"], columns=["property"])
    .filter(pl.col("Capacity Curtailed").gt(0))
    .sort(pl.col("datetime"))
)

datetime,generator,Generation,Units Generating,Capacity Curtailed,Max Capacity,Available Capacity
datetime[μs],str,f64,f64,f64,f64,f64
2024-07-02 00:00:00,"""EL_NOGAL_EO""",0.0,1.0,4.03236,9.0,4.03236
2024-07-02 00:00:00,"""LOS_OLMOS_EO""",0.0,1.0,55.73,100.0,55.73
2024-07-02 00:00:00,"""MALLECO_SUR_EO…",66.943131,1.0,32.446869,135.1,99.39
2024-07-02 01:00:00,"""CAMPO_LINDO_EO…",0.0,1.0,44.52,67.4,44.52
2024-07-02 01:00:00,"""EL_NOGAL_EO""",0.0,1.0,4.11516,9.0,4.11516
2024-07-02 01:00:00,"""MALLECO_NORTE_…",72.537287,1.0,18.962713,137.9,91.5
2024-07-02 01:00:00,"""RENAICO_EO""",0.0,1.0,41.05,88.0,41.05
2024-07-02 02:00:00,"""EL_NOGAL_EO""",0.0,1.0,4.36356,9.0,4.36356
2024-07-02 02:00:00,"""MALLECO_SUR_EO…",77.701617,1.0,2.7183834,135.1,80.42


In [21]:
(
    procesor_cdc.data
    #.group_by(pl.col("datetime")).agg(pl.col("Capacity Curtailed").sum())
    .sort(pl.col("datetime"))
 )

node,datetime,generator,Generation,Capacity Curtailed,Max Capacity,Available Capacity,Prorrata,Error
str,datetime[μs],str,f64,f64,f64,f64,f64,f64
"""Arica066""",2024-07-02 14:00:00,"""CHAPIQUINA""",3.3,0.0,10.14,3.3,3.3,0.000011
"""Maitencillo220…",2024-07-02 14:00:00,"""ATACAMA_EO""",125.5,0.0,165.3,125.5,118.143894,0.0
"""Maitencillo220…",2024-07-02 14:00:00,"""CABO_LEONES_1_…",37.523,0.0,115.5,37.523,37.523,0.000127
"""Maitencillo220…",2024-07-02 14:00:00,"""CABO_LEONES_2_…",46.45,0.0,205.7,46.45,46.45,0.000227
"""Maitencillo220…",2024-07-02 14:00:00,"""CABO_LEONES_3_…",56.9,0.0,190.2,56.9,56.9,0.00021
"""Salar220""",2024-07-02 14:00:00,"""CALAMA_EO""",14.31,0.0,150.0,14.31,14.31,0.000165
"""Ohiggins220_BP…",2024-07-02 14:00:00,"""CERRO_TIGRE_EO…",69.888,0.0,184.8,69.888,69.888,0.000204
"""Parinas500""",2024-07-02 14:00:00,"""HORIZONTE_EO""",13.7,0.0,821.0,13.7,13.7,0.000905
"""Ohiggins220_BP…",2024-07-02 14:00:00,"""LLANOS_DEL_VIE…",41.2,0.0,156.1,41.2,41.2,0.000172


In [2]:
import polars as pl
from prorrata.transform_cdc import DataProcessor as CDC
from prorrata.transform import DataProcessor as ORI

procesor_cdc = CDC.from_extractor(data)
procesor_cdc.process_prorrata()

procesor_ori = ORI(data)
procesor_ori.process_prorrata()

In [ ]:
procesor_cdc.get_t_data(data)
procesor_cdc.t_data_0.collect()

In [5]:
compare_data = (
    procesor_cdc.data.lazy().select(["node","datetime","generator","Generation","Max Capacity", "Available Capacity", "Prorrata"])
    .join(
        procesor_ori.data.select(["node","datetime","generator","Prorrata"]),
        on=["node","datetime","generator"],
        how="inner"
    )
    .rename({"Prorrata": "Prorrata_cdc", "Prorrata_right": "Prorrata_ori"})
    .collect()
)
compare_data

node,datetime,generator,Generation,Max Capacity,Available Capacity,Prorrata_cdc,Prorrata_ori
str,datetime[μs],str,f64,f64,f64,f64,f64
"""Arica066""",2024-07-02 14:00:00,"""CHAPIQUINA""",3.3,10.14,3.3,3.3,3.100601
"""Arica066""",2024-07-02 15:00:00,"""CHAPIQUINA""",3.3,10.14,3.3,3.3,3.017954
"""Maitencillo220…",2024-07-02 14:00:00,"""ATACAMA_EO""",125.5,165.3,125.5,118.143894,122.249438
"""Maitencillo220…",2024-07-02 15:00:00,"""ATACAMA_EO""",115.878,165.3,115.878,113.682718,111.280152
"""Maitencillo220…",2024-07-02 14:00:00,"""CABO_LEONES_1_…",37.523,115.5,37.523,37.523,35.251736
"""Maitencillo220…",2024-07-02 15:00:00,"""CABO_LEONES_1_…",39.79749,115.5,39.79749,39.79749,36.584838
"""Maitencillo220…",2024-07-02 14:00:00,"""CABO_LEONES_2_…",46.45,205.7,46.45,46.45,42.404987
"""Maitencillo220…",2024-07-02 15:00:00,"""CABO_LEONES_2_…",49.6434,205.7,49.6434,49.6434,43.921819
"""Maitencillo220…",2024-07-02 14:00:00,"""CABO_LEONES_3_…",56.9,190.2,56.9,56.9,53.159789


In [6]:
from datetime import datetime
(
    compare_data
    .with_columns(
        (pl.col("Available Capacity") / pl.col("Max Capacity")).alias("Porc. Available"),
        (pl.col("Prorrata_ori") / pl.col("Max Capacity")).alias("Porc. ori"),
        (pl.col("Prorrata_cdc") / pl.col("Max Capacity")).alias("Porc. cdc"),
        
    )
    #.filter(pl.col("generator").eq("CHAPIQUINA"))
    #.filter(pl.col("datetime").eq(datetime(2024,2,14,10)))
    #.filter(pl.col("Prorrata_ori").lt(pl.col("Prorrata_cdc")))
    .with_columns(
        pl.when(pl.col("Porc. ori").gt(pl.col("Porc. cdc"))).then(True).otherwise(False).alias("Test"),

    )
    #.filter(pl.col("Max Capacity").lt(pl.col("Available Capacity").max().over("generator")))
    #.filter(pl.col("Available Capacity").max())
    #.sort("datetime")
    #.head(30)
    .write_csv("compare_data.csv", datetime_format="%Y-%m-%d %T")
)

In [7]:
(
    procesor_cdc.data.lazy()
    #.filter(pl.col("generator").eq("CHAPIQUINA"))
    .group_by("datetime")
    .agg(
        Sum_Gen = pl.sum("Generation"),
        Sum_Available = pl.sum("Available Capacity"), 
        Sum_Prorrata = pl.sum("Prorrata"),
        Sum_Error = pl.sum("Error"),
        Error = pl.sum("Prorrata") - pl.sum("Generation")
    )
    .sort("datetime")
    .collect()
)

datetime,Sum_Gen,Sum_Available,Sum_Prorrata,Sum_Error,Error
datetime[μs],f64,f64,f64,f64,f64
2024-07-02 14:00:00,3906.335803,4237.85248,3906.330009,0.005794,-0.005794
2024-07-02 15:00:00,3966.048038,4360.138126,3966.047088,0.00095,-0.00095


In [8]:
(
    procesor_ori.data
    #.filter(pl.col("generator").eq("CHAPIQUINA"))
    .group_by("datetime")
    .agg(
        Sum_Gen = pl.sum("Generation"),
        Sum_Available = pl.sum("Available Capacity"), 
        Sum_Prorrata = pl.sum("Prorrata"),
        Sum_Error = pl.sum("Error"),
        Error = pl.sum("Prorrata") - pl.sum("Generation")
    )
    .sort("datetime")
    .collect()
)

datetime,Sum_Gen,Sum_Available,Sum_Prorrata,Sum_Error,Error
datetime[μs],f64,f64,f64,f64,f64
2024-07-02 14:00:00,3906.335803,4237.85248,4088.187832,0.000029,181.852029
2024-07-02 15:00:00,3966.048038,4360.138126,4155.822168,0.000131,189.774131


In [9]:
(
    procesor_ori.data
    .filter(
        pl.col("Available Capacity").gt(pl.col("Max Capacity"))
    )
    .select(
        pl.col("generator").unique(),
    )
    .collect()
)

generator
str
"""POZO_ALMONTE_3…"
"""FINIS_TERRAE_F…"


# PRUEBA DE CASOS ERROR

Que pasa si no hay cmg menor a 0? o no hay curtailment? ... armar caso apra esto.

In [ ]:
from pathlib import Path
from prorrata.extract import DataExtractor


path_prg = r"C:\Users\felipe.bastidas\PyProyectos\Test_data\PRG20240214"
PATH_PMGD_EXCLUDE =  r"C:\Users\felipe.bastidas\PyProyectos\Test_data\Otros\Lista_PMGDs.xlsx"
PATH_BANNED_GENERATORS = r"C:\Users\felipe.bastidas\PyProyectos\Test_data\Otros\Centrales_Vetadas.xlsx"
PATH_ACCDB_INPUT = r"Datos/Model PRGdia_Full_Definitivo Solution/Model PRGdia_Full_Definitivo Solution.accdb"

data = DataExtractor(
    Path(path_prg).joinpath(PATH_ACCDB_INPUT),
    Path(PATH_PMGD_EXCLUDE),
    Path(PATH_BANNED_GENERATORS)
)

data.extract_data()

In [ ]:
data.check_curtailment()

In [ ]:
import polars as pl

(
    data.gen
    .filter(pl.col("property").eq("Capacity Curtailed"))
    .select(pl.col("value").sum())
)

In [ ]:
from prorrata.transform import DataProcessor

row, _ = data.cmg.shape

if row == 0:
    raise ValueError("No hay curtailment")

data_processor = DataProcessor(data)
data_processor.process_prorrata()